<a href="https://colab.research.google.com/github/dilvinagachanli/colab-rag-training/blob/main/colab_rag_poc_v_final_veri_dahil_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
# Cell 1: Check if the GPU is active
# Run this cell. If it's working, you should see a table with "T4" listed as the GPU.
!nvidia-smi

#%%
# Cell 2: Install all dependencies
# We are installing ollama, chromadb (for our vector DB), and sentence-transformers (for embeddings)
!pip install ollama chromadb sentence-transformers pandas

#%%
# Cell 3: Install and start the Ollama server (The "Engine")
# This will download and set up the Ollama server application inside Colab.
!curl -fsSL https://ollama.com/install.sh | sh

# Start the server in the background
import subprocess
import time

print("Attempting to start Ollama server in the background...")
# Run the server in a way that doesn't block the notebook
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
print("Ollama server should be running.")
# Give it a moment to start up
time.sleep(5)

# --- NEW CELL FOR RESTARTING ---
#%%
# Cell 3.5: (Run this IF you get a "Connection Error")
# This cell just restarts the Ollama server process without re-installing anything.
import subprocess
import time

print("Attempting to (re)start the Ollama server in the background...")
# Kill any existing ollama process
try:
    subprocess.run(["pkill", "-f", "ollama serve"], check=True)
    print("Old Ollama process stopped.")
except subprocess.CalledProcessError:
    print("No old Ollama process was found running.")

# Start the server in a way that doesn't block the notebook
ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
print("✅ Ollama server should be running again.")
time.sleep(5)
# --- END NEW CELL ---


#%%
# Cell 4: Step 1 - Pull the "Brain" (Our LLM)
# We are now focusing only on llama3:8b
!ollama pull llama3:8b

#%%
# Cell 5: Step 2 - Create the "Book" (Vector Database + Data)
# *** THIS IS THE NEW VERSION WITH EMBEDDED DATA ***

import chromadb
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb import EmbeddingFunction

# 1. Define our new, "smarter" multilingual embedding model
embedding_model_name = 'paraphrase-multilingual-mpnet-base-v2'
print(f"\nLoading embedding model: {embedding_model_name}...")
embedding_model = SentenceTransformer(embedding_model_name)
print("Embedding model loaded.")

# 2. Create a ChromaDB-compatible wrapper class to fix the 'name' attribute error
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model, name):
        self.model = model
        self._name = name

    def __call__(self, texts):
        # ChromaDB might pass a single string or a list. Handle both.
        if isinstance(texts, str):
            texts = [texts]
        return self.model.encode(texts).tolist()

    def name(self):
        # This is the method ChromaDB was looking for
        return self._name

# Create an instance of our new embedding function class
my_ef = MyEmbeddingFunction(embedding_model, embedding_model_name)

# 3. Define the ChromaDB client
db = chromadb.PersistentClient(path="./rag_db")

# 4. IMPORTANT: Delete the old, broken collection to fix the dimension mismatch
try:
    db.delete_collection(name="company_docs")
    print("\nOld 'company_docs' collection deleted.")
except Exception as e:
    print(f"\nNo old collection to delete (or other error): {e}")

# 5. Create a new collection *using the correct model*
print("Creating new 'company_docs' collection...")
collection = db.get_or_create_collection(
    name="company_docs",
    metadata={"hnsw:space": "cosine"}, # Use cosine similarity
    embedding_function=my_ef # Pass the class instance
)

# 6. Load your full, new dataset directly as a Python dictionary
print("\nLoading new dataset...")
data = {
    "text": [
        "Personel servislerinin kalkış saati kış tarifesi nedeniyle 18:15'e alınmıştır.", # doc_0
        "Tedarikçi 'Koroğlu Metal' ile yaptığımız 1.2 Milyon TL'lik alüminyum anlaşması onaylandı.", # doc_1
        "Q4 için stratejik hedefimiz, 'Pro-Screwdriver' serisinde %8 pazar payı kazanmaktır.", # doc_2
        "C Blok yemekhanesinde bugünkü menü: Mercimek çorbası, orman kebabı, pilav.", # doc_3
        "Ahmet D.'nin 15 günlük babalık izni İK tarafından onaylanmıştır.", # doc_4
        "Sevkiyat departmanı, 89564 nolu siparişin paletlemesini tamamladı.", # doc_5
        "Müşteri 'YapıMark'a 5000 adet 'Power Pense' için verdiğimiz birim fiyat 12.50 USD'dir. Bu fiyat gizlidir.", # doc_6
        "Yeni hidrolik presin kurulumu tamamlandı, Pazartesi günü devreye alınıyor.", # doc_7
        "Ayşe K.'nin maaşına %15 zam yapılması ve 120.000 TL prim ödenmesi kararlaştırıldı.", # doc_8
        "Ar-Ge ekibi, yeni tork anahtarının prototipini tamamladı. Kalıp maliyeti 80.000 EUR.", # doc_9
        "Fabrika genelinde 'İş Güvenliği Farkındalık' posterleri asılmıştır.", # doc_10
        "Hasan G. (Doğum Tarihi: 10.04.1985, TCKN: 12345678910) işe giriş evrakları tamamlandı.", # doc_11
        "Bu ayki enerji giderimiz 620.000 TL olarak faturalandırıldı, bütçenin %10 üzerinde.", # doc_12
        "2025 bütçesinde AR-GE payının %12'ye çıkarılması yönetim kuruluna sunulacak.", # doc_13
        "Otoparktaki 15 numaralı alana bir araç yanlış park etmiş, anons yapılacak.", # doc_14
        "Üretim bandındaki 'Isıl İşlem' prosesinin sıcaklık ayarı 850°C'den 865°C'ye çıkarıldı. Bu bir ticari sırdır.", # doc_15
        "Şirketimizin 3. Çeyrek cirosu 14.2 Milyon TL, net kârı 1.8 Milyon TL'dir.", # doc_16
        "Hafta sonu gönüllü mesaiye kalacak personelin listesi güvenlik birimine iletildi.", # doc_17
        "Tedarikçi 'Bilya Rulman A.Ş.'nin banka bilgileri: TR10 0001 0000 1234 5678 9012.", # doc_18
        "Yarınki denetim için lütfen kişisel dolaplarınızı kilitli tutunuz.", # doc_19
        "Satış ekibinin 'Anadolu Distribütörleri' ile yaptığı toplantının detaylı raporu ektedir.", # doc_20
        "Ofis klimalarının bakımı Cuma günü yapılacaktır.", # doc_21
        "Satılan malın maliyeti (SMM) bu ay hammadde zammı nedeniyle %5 arttı.", # doc_22
        "Proje 'Altın Pergel'in lansman tarihi rakip analizi nedeniyle 2 ay ertelendi.", # doc_23
        "Genel müdürün yeni e-posta imzası formatı IT tarafından ayarlanmıştır.", # doc_24
        "Sipariş No 90123: Müşteri 'Global İnşaat', 1000 adet çekiç, toplam bedel 15.000 EUR.", # doc_25
        "Yeni 'Anti-Pas' kaplama formülümüzün detayları sadece Ar-Ge ekibiyle paylaşılmalıdır.", # doc_26
        "Tesisimizin güney cephesindeki alarm sisteminin detaylı planı ve kör noktaları.", # doc_27
        "Zeynep Hanım'ın avans talebi (15.000 TL) finans departmanı tarafından reddedildi.", # doc_28
        "Yeni 'Pro-Grip' serisi el aletlerimizin kataloğu baskıdan geldi.", # doc_29
        "İşçilik giderleri, asgari ücret artışı nedeniyle aylık 450.000 TL ek maliyet getirdi.", # doc_30
        "Tedarikçi 'Yılmaz Plastik'in son sevkiyatındaki %8'lik fire oranı kabul edilemez.", # doc_31
        "Şirketimizin nakit akış tablosu, banka kredisi ödemesi nedeniyle bu ay 500 bin TL negatifte.", # doc_32
        "Haftalık 'Kalite Kontrol' toplantısı yarın 10:00'da.", # doc_33
        "Doğum günü kutlaması için mola odasında pasta kesilecektir.", # doc_34
        "Elif S.'nin kan grubu B+, acil durum kartı güncellendi.", # doc_35
        "CNC tezgahlarının yeni 'toolpath' optimizasyon süreci, parça başına 12 saniye kazandırdı.", # doc_36
        "Satış ekibinin bu ayki 2 Milyon TL'lik hedefini %90 oranında tutturduğunu görüyoruz.", # doc_37
        "Kantin işletmecisi ile olan sözleşmemiz 1 yıl daha uzatıldı.", # doc_38
        "Müşteri 'Teknik Hırdavat'ın 450.000 TL'lik vadesi geçmiş borcu için yasal takip başlatılacak.", # doc_39
        "Üretim tesisimizin yıllık 15.000 ton çelik işleme kapasitesi bilgisi gizlidir.", # doc_40
        "Bu yıl 'Yılın Çalışanı' ödülü için oylama başlamıştır.", # doc_41
        "ISO 9001 denetiminde 2 adet 'minör' uygunsuzluk tespit edildi, rapor ektedir.", # doc_42
        "Stratejik hedefimiz, 2026'da Orta Doğu pazarına girerek 5 Milyon USD ihracat geliri elde etmektir.", # doc_43
        "Yarın sabah 08:00'de genel bir elektrik kesintisi planlanmaktadır.", # doc_44
        "Yeni tedarikçimiz 'Aşındırıcı Tech'in vergi levhası ve ticari sicil bilgileri ektedir.", # doc_45
        "Bayram öncesi avans ödemeleri Cuma günü hesaplara yatacaktır.", # doc_46
        "Dökümhanedeki A-3 Pota'nın değiştirilmesi için 25.000 EUR bütçe onaylandı.", # doc_47
        "Aylık genel üretim giderleri (GÜG) içinde 'amortisman' payı 110.000 TL'dir.", # doc_48
        "Şirketimizin logosu web sitesinde güncellenmiştir.", # doc_49
        "Üretim maliyet analizine göre, 'Model-X' pensesinin birim maliyeti 4.80 EUR'dur.", # doc_50
        "Yusuf Bey'in babasının vefatı nedeniyle başsağlığı dileriz. Cenaze bilgileri ektedir.", # doc_51
        "İK Departmanı, tüm personelin 'Performans Değerlendirme' formlarını doldurmasını rica eder.", # doc_52
        "Tesisin çatısına kurulacak GES (Güneş Enerjisi Sistemi) için 4 firmadan teklif alındı.", # doc_53
        "Depodaki forkliftin periyodik bakımı yapılmıştır.", # doc_54
        "Müşteri 'Süper Marketler Zinciri' ile olan özel markalı (private label) ürün sözleşmesi imzalandı.", # doc_55
        "Satış müdürünün aylık 180.000 TL olan maaşı, 210.000 TL'ye yükseltildi.", # doc_56
        "Fabrika bahçesindeki çimler biçildi.", # doc_57
        "Tedarikçi 'Mavi Lojistik' ile olan navlun anlaşmamız, km başına 0.85 USD olarak güncellendi.", # doc_58
        "EBITDA marjımız bu çeyrek %14.5 olarak gerçekleşti, bütçenin 1 puan altında.", # doc_59
        "Hafta sonu IT ekibi sunucu güncellemesi yapacaktır, sistemlere erişim olmayabilir.", # doc_60
        "Sipariş No 89771: 50 adet 'Ağır Hizmet Çekici', 12.000 TL, müşteriye faturalandı.", # doc_61
        "Personelimizden Ali V.'nin anne adı 'Fatma', baba adı 'Mehmet'tir.", # doc_62
        "Yaz saati uygulaması nedeniyle mesai başlangıcı 07:30'a çekilmiştir.", # doc_63
        "Satın alma ekibi, 'Endüstriyel Yağ' alımı için 'Petrol Ofisi'nden %5 özel indirim aldı.", # doc_64
        "Bu ayki işçilik giderleri, fazla mesailer nedeniyle 850.000 TL olarak gerçekleşti.", # doc_65
        "Tüm çalışanların 'Acil Durum Toplanma Alanı' konusunda bilgi sahibi olması gerekmektedir.", # doc_66
        "Müşteri 'X İnşaat'ın 1.2 Milyon TL'lik siparişinin %30'u peşin tahsil edildi.", # doc_67
        "Proje 'Titanyum' (yeni nesil pense serisi) için patent başvurusu yapıldı.", # doc_68
        "Şirketimizin 2025 yılı 350 Milyon TL ciro bütçesi onaylanmıştır.", # doc_69
        "3 numaralı montaj hattında kalite kontrol noktası eklenmiştir.", # doc_70
        "Müşteri 'Almanya Distribütörü GmbH' ile olan münhasırlık (exclusivity) sözleşmemiz 2 yıl uzatıldı.", # doc_71
        "Dolar kurundaki artış nedeniyle hammadde alış maliyetimiz %9 arttı.", # doc_72
        "A Blok'taki kahve makinesi arızalanmıştır, teknik servis çağrıldı.", # doc_73
        "Üretim prosesimizdeki 'soğuk dövme' tekniği, rakiplerimize göre %15 daha az malzeme kullanmamızı sağlıyor.", # doc_74
        "Personelimizden Emre T.'nin banka hesabı (IBAN: TR40...) maaş ödemesi için güncellendi.", # doc_75
        "Yeni yıl kutlaması için Cuma günü 16:00'da bahçede toplanılacaktır.", # doc_76
        "Tedarikçi 'Hızlı Kargo' ile olan anlaşmamız, 'gecikme cezası' maddesi eklenerek yenilendi.", # doc_77
        "Tesisimizin aylık 400.000 kWh olan enerji tüketimini azaltmak için bir proje başlatıldı.", # doc_78
        "Web sitemizin 'Kariyer' sayfası güncellendi, yeni ilanlar eklendi.", # doc_79
        "Satış ekibinin 'Bölge Bayi Toplantısı' sunumundaki ciro ve kârlılık verileri gizlidir.", # doc_80
        "Ayın 5'inde ödenen 120.000 TL'lik vergi (KDV) beyannamesi ektedir.", # doc_81
        "Misafir otoparkı alanına malzeme sevkiyatı yapılmayacaktır.", # doc_82
        "İK'nın yaptığı 'Çalışan Memnuniyet Anketi' sonuçlarına göre maaş skalası en düşük puanı aldı.", # doc_83
        "Sipariş 89800: 200 adet 'İzoleli Tornavida Seti', üretim planına alındı.", # doc_84
        "1 Mayıs Emek ve Dayanışma Günü'nde tesisimiz kapalı olacaktır.", # doc_85
        "Müşteri 'Yurt Dışı Holding'e gönderilen fiyat teklifi No. 2025-045 onay bekliyor.", # doc_86
        "Yeni pres makinesinin 4.5 Milyon TL'lik yatırım maliyeti, 18 ayda geri dönecek (ROI).", # doc_87
        "Toplantı odasındaki projeksiyon cihazının lambası değiştirildi.", # doc_88
        "İK veritabanı: 'Murat Ç.', Borç: 5.000 TL (avans), Alacak: 0 TL, Hakediş: 95.000 TL.", # doc_89
        "Nakit akışımızı düzenlemek için 'Çek Ödemeleri' 60 günden 90 güne çıkarıldı.", # doc_90
        "Bugünkü öğle yemeği için dışarıdan sipariş vermek serbesttir.", # doc_91
        "Toplam kapalı üretim alanımız 15.000 m2 olup, Türkiye'nin en büyük entegre el aleti üreticisiyiz.", # doc_92
        "Aliağa'daki yeni 10.000 m2'lik tesisin yerleşim planında (layout), CNC hattı ile montaj hattı yan yana konumlandırıldı.", # doc_93
        "Bir adet 'Pro-Pense'nin üretimi, başından sonuna kadar tam 45 farklı operasyondan geçer.", # doc_94
        "Pense üretim akışı: 1. Şablon Kesme (Lazer), 2. Dövme (Sıcak), 3. Çapak Alma (Pres), 4. Kumlama, 5. Kalibre Etme.", # doc_95
        "Tüm anahtar serimizin ana hammaddesi, dayanıklılığı kanıtlanmış 31 CRV 3 çeliğidir.", # doc_96
        "31 CRV 3 hammaddesini 'A Metalurji' firmasından 75 TL/kg, 'B Çelik' firmasından ise 78 TL/kg maliyetle alıyoruz.", # doc_97
        "Tüm teknik resimler ve tasarım belgeleri, kalite standartlarımız gereği 10 yıl boyunca muhafaza edilir.", # doc_98
        "Ekteki '19mm Lokma Anahtarı' teknik resminde (Resim No: T-105-B), tork toleransının +/- 0.2 Nm olduğu belirtilmiştir.", # doc_99
        "Ar-Ge ekibimizin 'G-10' lokma anahtarı için hazırladığı teknik resmin tüm detayları sunucudadır.", # doc_100
        "Ayşe Hanım'ın (İK) aylık maaşı 110.000 TL, eşi Ahmet Bey (Doğum T: 1985) olarak kayıtlıdır.", # doc_101
        "Şirketimiz, sektörde 30. yılını kutluyor.", # doc_102
        "Aliağa'daki yeni fabrika yatırımının toplam bütçesi 15 Milyon EUR olarak onaylandı.", # doc_103
        "Ürettiğimiz bir pensenin yaklaşık 45 operasyonu vardır, bu da kaliteye verdiğimiz önemi gösterir.", # doc_104
        "'31 CRV 3' çeliğini 'A Firması'ndan alıyoruz çünkü B firmasına göre %5 daha ucuzlar (75 TL/kg).", # doc_105
        "Kalite departmanı, teknik resimlerin arşivlenme sürecini denetleyecektir (SOP Kodu: KLT-008).", # doc_106
        "Pensenin dövme operasyonundan sonra 'çapak alma' ve 'kumlama' işlemlerine girmesi, üretim prosesimizin bir parçasıdır.", # doc_107
        "Bu ayki ciro 12 Milyon TL, Satılan Malın Maliyeti (SMM) ise 7.8 Milyon TL'dir.", # doc_108
        "Toplam 15.000 m2'lik tesisimizde 500'den fazla çalışanımızla hizmet veriyoruz.", # doc_109
        "Yeni 'Aliağa' tesisinin layout planında, hammadde giriş kapısı ile pres hattı arasındaki mesafe optimize edildi.", # doc_110
        "31 CRV 3 hammaddesinin alternatifi olarak 42 CrMo4 çeliğini test ediyoruz, tedarikçisi 'C Firması'.", # doc_111
        "Kombine anahtarlarımızın standardı 31 CRV 3 çeliktir.", # doc_112
        "Ekteki 'Tornavida Sapı' (Parça No: 88-A) teknik resmine göre kullanılacak plastik hammaddesi PA6 %30 GF'dir.", # doc_113
        "Müşteri 'Global Hırdavat'a 500.000 TL'lik sipariş için %15 iskonto uygulandı.", # doc_114
        "Üretimimizdeki 45 operasyonun 10 tanesi tam otomatiktir.", # doc_115
        "Pense üretiminde 'kumlama' operasyonundan sonra 'kalibre' işlemi yapılması kritik önemdedir.", # doc_116
        "10.000 m2'lik yeni Aliağa fabrikamızın temel atma töreni gelecek ay.", # doc_117
        "Teknik resim No: L-204 (Lokma Anahtarı), 3D CAD dosyası ve malzeme listesi (BOM) ektedir.", # doc_118
        "Hammadde 31 CRV 3'ün ana tedarikçisi 'A Firması'dır. Alternatif 'B Firması' ile görüşmeler sürüyor.", # doc_119
        "Üretim tesisimizin toplam kapalı alanı 15.000 m2'dir.", # doc_120
        "Bu ayki işçilik giderleri 1.5 Milyon TL, enerji giderleri ise 700.000 TL olarak gerçekleşti.", # doc_121
        "Tüm teknik resimler, tasarım ekibi tarafından 'SolidWorks PDM' sisteminde 10 yıl saklanır.", # doc_122
        "Pense üretimi: 1.Dövme, 2.Kumlama, 3.Çapak Alma. Bu sıra yanlıştır, doğru sıra: Dövme -> Çapak -> Kumlama.", # doc_123
        "Ahmet Y.'nin (Mühendis) baba adı 'Hasan', doğum yeri 'İzmir', kan grubu 'A+'.", # doc_124
        "Anahtar üretiminde 31 CRV 3 çeliği kullanıyoruz.", # doc_125
        "Aliağa tesisinin layout çizimleri, otomasyon entegrasyonu için 'Siemens' firmasına gönderildi.", # doc_126
        "15.000 m2'lik fabrikamızda günde 20.000 adet el aleti üretiyoruz.", # doc_127
        "31 CRV 3 hammaddesi için 'A Firması' ile 75 TL/kg üzerinden 1 yıllık 1000 tonluk sözleşme imzaladık.", # doc_128
        "Bir adet pense 45 operasyon görür; bir tornavida ise sadece 20 operasyon.", # doc_129
        "Teknik resim T-106'nın revizyonu: 'Lokma ağzı' çapı 22mm'den 22.1mm'ye çıkarıldı. Acil güncelleyin.", # doc_130
        "Pense üretimindeki 'kumlama' ve 'kalibre' operasyonları arasındaki süre 1 saati geçmemelidir.", # doc_131
        "Mustafa K.'nin (Operatör) avans talebi (20.000 TL) İK tarafından onaylandı.", # doc_132
        "Ürünlerimiz 31 CRV 3 çeliğinden üretildiği için paslanmaya karşı ömür boyu garantilidir.", # doc_133
        "Tedarikçi 'A Firması'ndan (31 CRV 3) 75 TL/kg fiyatı alamayan 'C Firması' (78 TL/kg) alımdan çıkarıldı.", # doc_134
        "15.000 m2'lik üretim alanımızın 2.000 m2'si Ar-Ge'ye ayrılmıştır.", # doc_135
        "Teknik resimlerin 10 yıl saklanması, ISO 9001 standardı gereğidir.", # doc_136
        "Aliağa'daki yeni tesisin 'dövme presi' temeli için 5 metre derinliğinde kazı yapılması gerekiyor.", # doc_137
        "Pense üretim hattındaki 45 operasyonun 'kritiklik' analizi (FMEA) ektedir.", # doc_138
        "Yemekhane menüsünde bu hafta Ege otları var.", # doc_139
        "'31 CRV 3' hammaddesinin tedarikçisi 'A Firması'nın sahibi 'Ali V.'dir.", # doc_140
        "Lokma anahtarı teknik resmindeki 'pah' açısı 45 derece olarak güncellendi.", # doc_141
        "Fabrikamız 15.000 m2'lik dev bir alanda yer almaktadır.", # doc_142
        "Üretim prosesi: 'Dövme' (1100°C) -> 'Çapak Alma' (Soğuk Pres) -> 'Kumlama' (Çelik Bilya).", # doc_143
        "Teknik resim T-108'in 10 yıl saklanması gerekiyor.", # doc_144
        "Aliağa tesisinin 'Yerleşim Planı V3' ektedir. Lütfen gizli tutun.", # doc_145
        "Müşteri 'Tekzen'e 2 Milyon TL'lik sevkiyat için %10 özel indirim kodu tanımlandı.", # doc_146
        "Bir pensenin 45 operasyonu vardır ve hepsi 'sıfır hata' ile denetlenir.", # doc_147
        "31 CRV 3 çeliğinin kimyasal analizi: %0.31 Karbon, %1.1 Krom, %0.15 Vanadyum.", # doc_148
        "Bu ayki 600.000 TL'lik 'Genel Üretim Gideri' (GÜG) bütçeyi 50.000 TL aştı.", # doc_149
        "Açık anahtarlarımızın hammaddesi 'Krom Vanadyum 31 CRV 3'tür.", # doc_150
        "Teknik resim arşivimiz 10 yıllık veriyi içerir.", # doc_151
        "Pense üretimindeki 'Kumlama' operasyonunda '0.8 mm çelik bilya' kullanılması proses sırrıdır.", # doc_152
        "Aliağa tesisinin 'layout'u, 'yalın üretim' (lean) prensiplerine göre tasarlandı.", # doc_153
        "Satış ekibinin 2026 hedefi, 15.000 m2'lik tesis kapasitesini %95'e çıkarmaktır.", # doc_154
        "Satış raporu: '10x12 anahtar' bu ay 15.000 adet sattı (Hammadde: 31 CRV 3).", # doc_155
        "Şirket avukatı, 'Teknik Resim' (IP) sızıntısı ile ilgili bir dava açıyor.", # doc_156
        "Servis güzergahı 2 (Karşıyaka) 15 dakika gecikecektir.", # doc_157
        "Pense üretimindeki 45 operasyonun maliyet dökümü ektedir.", # doc_158
        "31 CRV 3'ü 75 TL/kg'dan alıyoruz. Eğer 80 TL olursa kârlılığımız %2 düşer.", # doc_159
        "15.000 m2'lik fabrikamızda 'ISO 14001 Çevre Yönetim Sistemi' uygulanmaktadır.", # doc_160
        "Teknik resim L-205'in 10 yıl sonra imha edilmesi gerekiyor.", # doc_161
        "Aliağa tesisinin 'robotik otomasyon' planı ve yatırım maliyeti (2.5 Milyon EUR) ektedir.", # doc_162
        "Üretim prosesi: 'Çapak alma' işleminden sonra 'ısıl işlem' yapılması kesinlikle yasaktır.", # doc_163
        "Müşteri 'Koçtaş'tan 500 adet '19mm Lokma Anahtarı' siparişi alındı.", # doc_164
        "Anahtarımızın hammaddesi 31 CRV 3'tür ve bu, kalite belgelerimizde yazar.", # doc_165
        "Teknik resim T-110 (Kargaburun Pense) üzerindeki 'tolerans' bilgileri güncellendi.", # doc_166
        "Aylık araç kira bedelimiz 50.000 TL'ye yükseldi (Tedarikçi: A Kiralama).", # doc_167
        "15.000 m2'lik tesisimizin güvenlik planı ve kamera yerleşimleri ektedir.", # doc_168
        "Pense üretimindeki 45 operasyonun 'çevrim süresi' (cycle time) ölçümleri yapıldı.", # doc_169
        "Hafta sonu mesaiye kalacakların listesi İK'ya bildirildi.", # doc_170
        "'31 CRV 3' hammaddesinin tedarikçisi 'A Firması'nın sözleşmesi 'kalite sorunları' nedeniyle feshedildi.", # doc_171
        "Teknik resimlerin 10 yıl saklanması prosedürü, IT departmanı tarafından yedeklenmektedir.", # doc_172
        "Aliağa tesisinin 'enerji altyapısı' planı ve trafo gücü (3 MW) bilgileri.", # doc_173
        "Pense üretim prosesindeki 'kalibre' operasyonunun 'ayar parametreleri' (setup sheet) ektedir.", # doc_1TA4
        "Satış raporu: '10x12 Anahtar' (31 CRV 3 çelik) bu ay 12.000 adet satarak hedefi yakalayamadı.", # doc_175
        "Yeni 'Pro-Grip' serimizin lansman tarihi 15 Aralık olarak belirlendi.", # doc_176
        "Lokma anahtarı teknik resminde belirtilen 'sertlik' değeri (55 HRC) ticari sırdır.", # doc_177
        "Tedarikçi 'B Firması'nın (31 CRV 3) 78 TL/kg'lık teklifi, 'A Firması' (75 TL/kg) nedeniyle reddedildi.", # doc_178
        "Müşteri şikayeti: 'Satın aldığım 10x12 anahtarın (31 CRV 3) ağzı sıyırdı.'", # doc_179
        "Tesisimiz 15.000 m2'dir. Aliağa'ya açılacak yeni tesis ise 10.000 m2 olacaktır.", # doc_180
        "Pense üretimindeki 'dövme' kalıbının (T-102 Kalıp) tasarım ömrü 100.000 adettir.", # doc_181
        "Teknik resim arşivimiz 10 yıl geriye dönük olarak taranabilir durumdadır.", # doc_182
        "Bu ayki 2.1 Milyon TL'lik 'Hammadde Alış Maliyeti' (31 CRV 3 dahil) bütçeyi %10 aştı.", # doc_183
        "Aliağa tesisinin layout planında 'kimyasal atık' depolama alanı kuzey cephesine taşındı.", # doc_184
        "Pense üretimindeki 'kumlama' operasyonunun birim maliyeti 0.15 EUR'dur.", # doc_185
        "15.000 m2'lik tesisimizin 'Mavi Yaka' personel sayısı 350'dir.", # doc_186
        "Anahtar üretiminde 31 CRV 3 kullanmamız, rakiplerimizden (4140 çelik kullanan) bizi ayırıyor.", # doc_187
        "Teknik resim T-112'nin (Alyan Anahtar) 10 yıllık saklama süresi dolmuş, imha edilecek." # doc_188
    ],
    "label": [
        "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "GENEL",
        "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "HASSAS", "HASSAS", "GENEL",
        "HASSAS", "HASSAS", "HASSAS", "GENEL", "GENEL", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS",
        "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL",
        "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL",
        "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS",
        "HASSAS", "GENEL",
        # Added Data Labels
        "GENEL", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS",
        "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS",
        "GENEL", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "GENEL",
        "HASSAS", "HASSAS", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "GENEL", "GENEL",
        "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS", "HASSAS", "HASSAS", "GENEL", "HASSAS",
        "GENEL", "HASSAS", "HASSAS", "HASSAS", "HASSAS", "HASSAS", "GENEL"
    ]
}


# Verify the data lists are of the same length before creating the DataFrame
if len(data["text"]) != len(data["label"]):
    print(f"ERROR: Data lists have different lengths! Text: {len(data['text'])}, Label: {len(data['label'])}")
else:
    print(f"Data lists are valid. Total items: {len(data['text'])}, Label items: {len(data['label'])}")
    df = pd.DataFrame(data)
    print(df.head())

    # 7. Add the data to the collection
    documents = df["text"].tolist()
    metadatas = [{"label": str(label)} for label in df["label"].tolist()] # Ensure label is a string
    ids = [f"doc_{i}" for i in range(len(documents))]

    print(f"\nTotal documents to load: {len(documents)}")

    # 8. Load the documents into ChromaDB
    # This will automatically use our 'my_ef' function to create the 768-dimension embeddings
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

    print("\n✅ New data successfully loaded into ChromaDB with new embeddings.")

#%%
# Cell 6: Step 3 - Test the final PoC (The "Orchestrator")
# This cell now only runs our chosen model (llama3:8b)

import ollama
import time

# This is the "Orchestrator" function.
# It takes a question, finds relevant context, and asks the "Brain" to answer.
def ask_my_bot(model_name, question):
    print(f"--- Testing Model: {model_name} ---")
    print(f"🤖 Query: {question}\n")

    # 1. Embed the user's question using our new embedding model
    # We must use the *same* model we used to build the database
    question_embedding = embedding_model.encode(question).tolist()

    # 2. Query the "Book" (ChromaDB)
    # Find the top 2 most relevant documents
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=2,
    )

    found_context = "\n".join(results['documents'][0])
    found_labels = results['metadatas'][0]

    print("📚 Found context:")
    print(found_context)
    print(f"🔒 Metadata (Labels): {found_labels}\n")

    # 3. Define the prompt for the "Brain" (LLM)
    # We instruct it to use our context and speak Turkish
    system_prompt = """
    Sen bir şirket asistanısın. Kullanıcının sorusunu SADECE ve SADECE sana verilen 'BAĞLAM' bölümündeki bilgileri kullanarak cevapla.
    Eğer cevap bağlamda yoksa, 'Üzgünüm, bu bilgiye sahip değilim.' de.
    Cevaplarını her zaman Türkçe ver.
    """

    user_prompt = f"""
    BAĞLAM:
    {found_context}

    SORU:
    {question}
    """

    # 4. Ask the "Brain" (Ollama)
    print(f"🧠 Asking {model_name}...")
    try:
        start_time = time.perf_counter() # Start the timer

        response = ollama.chat(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )

        end_time = time.perf_counter() # Stop the timer
        time_taken = end_time - start_time

        print(f"\n✅ Answer:\n{response['message']['content']}")
        print(f"⏱️ Time taken: {time_taken:.4f} seconds")

        # Add a 10-second cooldown to let Ollama unload the model from VRAM
        print("Cooling down for 10 seconds...")
        time.sleep(10)

    except Exception as e:
        print(f"\n❌ Error calling {model_name}: {e}")

# --- Our Test Questions ---
test_questions = [
    "Personel servislerinin kalkış saati nedir?",
    "Ayşe K.'nin maaşına ne kadar zam yapıldı?",
    "C Blok yemekhanesinde bugün ne var?",
    "Fabrika logosu nerede güncellendi?",
    "31 CRV 3 çeliğini hangi firmadan alıyoruz?", # New test query
    "Pense üretim akışı nedir?" # New test query
]

print("====== STARTING RAG PoC TEST ======")

for question in test_questions:
    print("="*50)
    print(f"QUESTION: {question}")
    print("="*50)
    ask_my_bot("llama3:8b", question) # We are only testing llama3:8b now
    print("-"*20)

print("\n====== RAG PoC TEST COMPLETE ======")


#%%
# Cell 7: Step 4 - The "Retrieval Accuracy" Test
# This checks if the "Orchestrator" (RAG) is finding the
# *correct* document for our questions. This is our most important metric.

print("====== STARTING RETRIEVAL ACCURACY TEST ======")

# This is our "Ground Truth"
# We define the question and the *exact text* it should find.
# This is more robust than using doc_ids, which change.
ground_truth_text = {
    "Personel servis saati nedir?": "Personel servislerinin kalkış saati kış tarifesi nedeniyle 18:15'e alınmıştır.",
    "Ahmet D.'nin izni ne oldu?": "Ahmet D.'nin 15 günlük babalık izni İK tarafından onaylanmıştır.",
    "Güney cephesindeki alarm planı nedir?": "Tesisimizin güney cephesindeki alarm sisteminin detaylı planı ve kör noktaları.",
    "Zeynep Hanım'ın avans talebi ne oldu?": "Zeynep Hanım'ın avans talebi (15.000 TL) finans departmanı tarafından reddedildi.",
    "Kantin sözleşmesi ne oldu?": "Kantin işletmecisi ile olan sözleşmemiz 1 yıl daha uzatıldı.",
    "ISO 9001 denetiminde ne bulundu?": "ISO 9001 denetiminde 2 adet 'minör' uygunsuzluk tespit edildi, rapor ektedir.",
    "Yusuf Bey'in babasına ne oldu?": "Yusuf Bey'in babasının vefatı nedeniyle başsağlığı dileriz. Cenaze bilgileri ektedir.",
    "Elif S.'nin kan grubu nedir?": "Elif S.'nin kan grubu B+, acil durum kartı güncellendi.",
    "31 CRV 3 çeliğinin maliyeti nedir?": "31 CRV 3 hammaddesini 'A Metalurji' firmasından 75 TL/kg, 'B Çelik' firmasından ise 78 TL/kg maliyetle alıyoruz.",
    "Teknik resimlerin saklanma süresi nedir?": "Tüm teknik resimler ve tasarım belgeleri, kalite standartlarımız gereği 10 yıl boyunca muhafaza edilir.",
    "Aliağa'daki yeni tesis kaç m2?": "Aliağa'daki yeni 10.000 m2'lik tesisin yerleşim planında (layout), CNC hattı ile montaj hattı yan yana konumlandırıldı.",
    "Pense üretimindeki operasyon sayısı nedir?": "Bir adet 'Pro-Pense'nin üretimi, başından sonuna kadar tam 45 farklı operasyondan geçer.",
    "Lokma anahtarı teknik resmindeki tolerans nedir?": "Ekteki '19mm Lokma Anahtarı' teknik resminde (Resim No: T-105-B), tork toleransının +/- 0.2 Nm olduğu belirtilmiştir.",
    "Mustafa K.'nin avans talebi onaylandı mı?": "Mustafa K.'nin (Operatör) avans talebi (20.000 TL) İK tarafından onaylandı.",
    "Hangi çelik 31 CRV 3'ün alternatifi?": "31 CRV 3 hammaddesinin alternatifi olarak 42 CrMo4 çeliğini test ediyoruz, tedarikçisi 'C Firması'."
}

# Create a reverse map from text to doc_id
# This runs *after* Cell 5 has loaded the data
try:
    text_to_id_map = {doc: f"doc_{i}" for i, doc in enumerate(documents)}
except NameError:
    # This handles if Cell 5 wasn't run, but we can't build the map.
    print("Error: 'documents' list not found. Please run Cell 5 first.")
    text_to_id_map = {}


# Build the final, robust test_cases dictionary
test_cases = {}
for question, text in ground_truth_text.items():
    if text in text_to_id_map:
        test_cases[question] = text_to_id_map[text]
    else:
        print(f"WARNING: Ground truth text not found in dataset: {text[:30]}...")

print(f"\nRunning {len(test_cases)} test cases...")

score = 0
total = len(test_cases)

for question, expected_id in test_cases.items():
    # 1. Embed the question
    question_embedding = embedding_model.encode(question).tolist()

    # 2. Query ChromaDB to find the #1 most relevant doc
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=1,
    )

    if not results['ids']:
        print(f"  [FAIL] Q: '{question[:30]}...' -> No results returned from DB.")
        continue

    found_id = results['ids'][0][0]
    found_doc = results['documents'][0][0]

    # 3. Compare to our "Ground Truth"
    if found_id == expected_id:
        print(f"  [PASS] Q: '{question[:30]}...' -> Found: '{found_id}'")
        score += 1
    else:
        print(f"  [FAIL] Q: '{question[:30]}...' -> Expected: '{expected_id}', but Found: '{found_id}'")
        print(f"         (Found doc: '{found_doc[:50]}...')")


print("\n====== RETRIEVAL ACCURACY COMPLETE ======")
print(f"Score: {score} / {total} correct")
if total > 0:
    print(f"Retrieval Accuracy: {(score / total) * 100:.1f}%")
else:
    print("No test cases were run.")

#%%
# Cell 8: Aşama 1 - Sentetik Eğitim Veri Seti Oluşturma
#
# Bu hücrede, 'llama3:8b' modelini kullanarak 189 dokümanımızın her biri için
# 3 adet sahte (sentetik) soru üreteceğiz. Bu, (soru, pozitif_cevap) çiftlerimizi
# oluşturarak 'fine-tuning' için eğitim setimizi hazırlayacak.

import ollama
import json
import pandas as pd
from tqdm.notebook import tqdm # İlerleme çubuğu (progress bar) için
import time
import re # JSON temizliği için RegEx kütüphanesini içe aktar

print("Sentetik eğitim veri seti oluşturma başlıyor...")

# Cell 5'te 'documents' listesinin oluşturulduğundan emin olalım
try:
    if not 'documents' in locals() or not 'metadatas' in locals() or not 'ids' in locals():
        raise NameError("'documents', 'metadatas', or 'ids' listesi bulunamadı. Lütfen önce Cell 5'i çalıştırdığınızdan emin olun.")
except NameError as e:
    print(f"HATA: {e}")
    # Bu hücrenin çalışmasını durdur
    raise

# Bu talimat (prompt), LLM'e nasıl soru üreteceğini öğretir
generator_system_prompt = """
Sen bir veri oluşturma asistanısın. TEK görevin, sana verilen bir 'BAĞLAM' metnini okumak ve bu bağlam tarafından DOĞRUDAN cevaplanabilecek 3 adet gerçekçi, yüksek kaliteli soru üretmektir.
- Soruları TÜRKÇE olarak üret.
- Bağlamda cevabı olmayan sorular sorma.
- SADECE ve SADECE geçerli bir JSON listesi (array) formatında cevap ver. Başka hiçbir metin ekleme.
- Örnek:
  BAĞLAM: "Personel servislerinin kalkış saati kış tarifesi nedeniyle 18:15'e alınmıştır."
  CEVAP:
  [
    "Personel servislerinin kalkış saati nedir?",
    "Servisler kış tarifesinde saat kaçta kalkıyor?",
    "Servis saati neden 18:15 oldu?"
  ]
"""

training_data = [] # Yeni (soru, pozitif_cevap) çiftlerimizi burada saklayacağız
failed_documents = 0

# 189 dokümanın tamamı üzerinden bir ilerleme çubuğu ile döngüye girelim
for doc in tqdm(documents, desc="Sorular üretiliyor"):

    user_prompt = f"""
    BAĞLAM: "{doc}"
    CEVAP:
    """

    try:
        # Sunucunun uyku moduna geçmemesi için her 10 dokümanda bir 'uyandır'
        if documents.index(doc) % 10 == 0:
            ollama.list() # Sunucuya hafif bir ping atar

        response = ollama.chat(
            model='llama3:8b', # "Beynimizi" bir jeneratör olarak kullanıyoruz
            messages=[
                {"role": "system", "content": generator_system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            # format='json' parametresini kullanmak yerine manuel olarak parse edeceğiz,
            # çünkü modelin hatalı JSON'unu ayıklamamız gerekiyor.
        )

        raw_text = response['message']['content']

        # --- YENİ SAĞLAM JSON AYIKLAYICI (ROBUST JSON PARSER) ---
        # Modelin cevabındaki ilk '[' ve son ']' arasındaki her şeyi bul
        json_match = re.search(r'\[.*\]', raw_text, re.DOTALL)

        if json_match:
            json_string = json_match.group(0)
            # Yarıda kesilmiş JSON'u onarmaya çalış
            if not json_string.endswith(']'):
                json_string += '"]' # Eksik kapanışı zorla ekle (basit bir onarım)

            # Tırnak işaretleriyle ilgili yaygın hataları düzelt
            json_string = json_string.replace(r'\",', '",').replace(r'\"', '"')

            try:
                questions = json.loads(json_string)
            except json.JSONDecodeError as json_err:
                # Eğer RegEx sonrası bile bozuksa, bu dokümanı atla
                print(f"\nUyarı: JSON parse hatası (onarım sonrası): {doc[:30]}... Hata: {json_err}")
                print(f"Alınan Ham Metin: {raw_text}")
                failed_documents += 1
                continue # Bir sonraki dokümana geç

            for q in questions:
                if q and isinstance(q, str): # Boş soru gelmediğinden ve string olduğundan emin ol
                    # (soru, pozitif_cevap) çiftini ekle
                    training_data.append({
                        "query": q,
                        "positive_passage": doc
                    })
        else:
            # Model hiç JSON listesi döndürmediyse
            print(f"\nUyarı: Cevapta JSON listesi bulunamadı: {doc[:30]}...")
            print(f"Alınan Ham Metin: {raw_text}")
            failed_documents += 1
        # --- JSON AYIKLAYICI SONU ---

    except Exception as e:
        print(f"\nKritik Hata: Bu doküman işlenemedi: {doc[:30]}... Hata: {e}")
        failed_documents += 1

    # Colab'ın GPU'sunu ve Ollama sunucusunu yormamak için kısa bir duraklama
    time.sleep(1) # Hızı biraz arttıralım

print("\n\nSentetik Veri Seti Oluşturma Tamamlandı!")
print(f"Başarıyla {len(training_data)} adet (soru/cevap) çifti oluşturuldu.")
print(f"{failed_documents} adet doküman işlenemedi.")

# Yeni eğitim setimizi bir DataFrame'e dönüştürelim
if training_data:
    df_train = pd.DataFrame(training_data)
    print(df_train.head())
else:
    print("UYARI: Hiç eğitim verisi oluşturulamadı. Fine-tuning işlemi atlanacak.")

#%%
# Cell 9: Aşama 2 - Embedding Modelini Eğitme (Fine-Tuning)
#
# Şimdi oluşturduğumuz bu sentetik veri setini kullanarak
# 'paraphrase-multilingual-mpnet-base-v2' modelini eğiteceğiz.
# Bu, modele bizim verilerimizin "anlamını" öğretecek.

from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader

# Sadece eğitim verisi oluşturulduysa bu hücreyi çalıştır
if 'df_train' in locals() and not df_train.empty:
    print(f"\nFine-tuning için temel model yükleniyor: '{embedding_model_name}'...")
    # 1. Eğitmek istediğimiz "temel" modeli yüklüyoruz
    #    (Bu, Cell 5'teki modelle aynı olmalı)
    model_to_train = SentenceTransformer(embedding_model_name)

    # 2. DataFrame'imizi 'InputExample' objelerine dönüştürüyoruz
    #    Bu, 'sentence-transformers' kütüphanesinin ihtiyaç duyduğu formattır
    train_examples = []
    for index, row in df_train.iterrows():
        train_examples.append(InputExample(texts=[row['query'], row['positive_passage']]))

    print(f"{len(train_examples)} adet eğitim örneği oluşturuldu.")

    # 3. "DataLoader" ve "Loss Function" tanımlanıyor
    #    Bu, modele *nasıl* öğreneceğini söyler
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

    # MultipleNegativesRankingLoss, bu tür "sorudan-dokümana" eşleştirme
    # fine-tuning'i için en etkili ve standart loss fonksiyonudur.
    train_loss = losses.MultipleNegativesRankingLoss(model=model_to_train)

    # 4. Eğitimi başlat!
    #    'epochs=1' tüm veri setimizin üzerinden bir kez geçeceğimiz anlamına gelir.
    #    Bu genellikle büyük bir doğruluk artışı için yeterlidir.
    num_epochs = 1
    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) # Adımların %10'u

    print("Model fine-tuning işlemi başlıyor... (Bu işlem 5-10 dakika sürebilir)")
    model_to_train.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=num_epochs,
        warmup_steps=warmup_steps,
        output_path='./my-finetuned-model-v1', # Bu, bizim yeni özel modelimiz
        show_progress_bar=True
    )

    print("\n✅ Fine-tuning tamamlandı! Yeni model './my-finetuned-model-v1' klasörüne kaydedildi.")
else:
    print("HATA: Eğitim verisi (df_train) bulunamadığı için Cell 9 (Fine-Tuning) atlandı.")


#%%
# Cell 10: Aşama 3 - YENİ Fine-Tune Edilmiş Modeli Değerlendirme
#
# Bu son adım. Cell 7'deki doğruluk testimizi yeniden çalıştıracağız,
# ancak bu kez YENİ eğittiğimiz modeli kullanarak
# %46.7'lik doğruluk skorunun artıp artmadığını göreceğiz.

import os

# Sadece fine-tuning işlemi başarıyla tamamlandıysa bu hücreyi çalıştır
finetuned_model_path = './my-finetuned-model-v1'
if 'df_train' in locals() and not df_train.empty and os.path.exists(finetuned_model_path):
    print("Yeni fine-tune edilmiş model yükleniyor...")
    # 1. Az önce kaydettiğimiz özel modeli yüklüyoruz
    finetuned_embedding_model = SentenceTransformer(finetuned_model_path)

    print("====== FİNAL DOĞRULUK TESTİ (Fine-Tuned Model ile) BAŞLIYOR ======")

    # 2. "Referans Veri" (Ground Truth) setimizi yeniden oluşturuyoruz
    ground_truth_text = {
        "Personel servis saati nedir?": "Personel servislerinin kalkış saati kış tarifesi nedeniyle 18:15'e alınmıştır.",
        "Ahmet D.'nin izni ne oldu?": "Ahmet D.'nin 15 günlük babalık izni İK tarafından onaylanmıştır.",
        "Güney cephesindeki alarm planı nedir?": "Tesisimizin güney cephesindeki alarm sisteminin detaylı planı ve kör noktaları.",
        "Zeynep Hanım'ın avans talebi ne oldu?": "Zeynep Hanım'ın avans talebi (15.000 TL) finans departmanı tarafından reddedildi.",
        "Kantin sözleşmesi ne oldu?": "Kantin işletmecisi ile olan sözleşmemiz 1 yıl daha uzatıldı.",
        "ISO 9001 denetiminde ne bulundu?": "ISO 9001 denetiminde 2 adet 'minör' uygunsuzluk tespit edildi, rapor ektedir.",
        "Yusuf Bey'in babasına ne oldu?": "Yusuf Bey'in babasının vefatı nedeniyle başsağlığı dileriz. Cenaze bilgileri ektedir.",
        "Elif S.'nin kan grubu nedir?": "Elif S.'nin kan grubu B+, acil durum kartı güncellendi.",
        "31 CRV 3 çeliğinin maliyeti nedir?": "31 CRV 3 hammaddesini 'A Metalurji' firmasından 75 TL/kg, 'B Çelik' firmasından ise 78 TL/kg maliyetle alıyoruz.",
        "Teknik resimlerin saklanma süresi nedir?": "Tüm teknik resimler ve tasarım belgeleri, kalite standartlarımız gereği 10 yıl boyunca muhafaza edilir.",
        "Aliağa'daki yeni tesis kaç m2?": "Aliağa'daki yeni 10.000 m2'lik tesisin yerleşim planında (layout), CNC hattı ile montaj hattı yan yana konumlandırıldı.",
        "Pense üretimindeki operasyon sayısı nedir?": "Bir adet 'Pro-Pense'nin üretimi, başından sonuna kadar tam 45 farklı operasyondan geçer.",
        "Lokma anahtarı teknik resmindeki tolerans nedir?": "Ekteki '19mm Lokma Anahtarı' teknik resminde (Resim No: T-105-B), tork toleransının +/- 0.2 Nm olduğu belirtilmiştir.",
        "Mustafa K.'nin avans talebi onaylandı mı?": "Mustafa K.'nin (Operatör) avans talebi (20.000 TL) İK tarafından onaylandı.",
        "Hangi çelik 31 CRV 3'ün alternatifi?": "31 CRV 3 hammaddesinin alternatifi olarak 42 CrMo4 çeliğini test ediyoruz, tedarikçisi 'C Firması'."
    }

    # (Cell 5'te tanımlanan 'documents' listesini kullandığımızdan emin olalım)
    text_to_id_map = {doc: f"doc_{i}" for i, doc in enumerate(documents)}
    test_cases = {}
    for question, text in ground_truth_text.items():
        if text in text_to_id_map:
            test_cases[question] = text_to_id_map[text]
        else:
            print(f"UYARI: Referans veri setindeki bu metin ana veri setinde bulunamadı: {text[:30]}...")

    print("\n--- Adil bir test için veritabanı YENİ fine-tune edilmiş modelle yeniden oluşturuluyor ---")

    # 3. Yeni eğittiğimiz modeli ChromaDB'nin anlayacağı formata getiriyoruz
    #    (Cell 5'teki 'MyEmbeddingFunction' sınıfını kullandığımızdan emin olalım)
    try:
        finetuned_ef = MyEmbeddingFunction(finetuned_embedding_model, "my-finetuned-model-v1")
    except NameError:
        print("HATA: 'MyEmbeddingFunction' sınıfı tanımlanmadı. Lütfen Cell 5'i yeniden çalıştırın.")
        raise

    # 4. Fine-tune edilmiş model için YENİ bir koleksiyon oluşturuyoruz
    try:
        db.delete_collection(name="company_docs_finetuned")
        print("Eski 'fine-tuned' koleksiyon silindi.")
    except Exception:
        pass # Zaten yoksa sorun değil

    ft_collection = db.get_or_create_collection(
        name="company_docs_finetuned",
        metadata={"hnsw:space": "cosine"},
        embedding_function=finetuned_ef
    )

    # 5. Tüm 189 dokümanı bu YENİ koleksiyona ekliyoruz
    ft_collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    print("Yeni fine-tune edilmiş veritabanı oluşturuldu.")

    print(f"\n--- YENİ Fine-Tuned Veritabanı üzerinde {len(test_cases)} adet test senaryosu çalıştırılıyor ---")
    score = 0
    total = len(test_cases)

    for question, expected_id in test_cases.items():
        # 6. Soruyu *yeni modelle* embed ediyoruz
        question_embedding = finetuned_embedding_model.encode(question).tolist()

        # 7. YENİ veritabanını sorguluyoruz
        results = ft_collection.query(
            query_embeddings=[question_embedding],
            n_results=1,
        )

        if not results['ids']:
            print(f"  [FAIL] S: '{question[:30]}...' -> DB'den sonuç dönmedi.")
            continue

        found_id = results['ids'][0][0]
        found_doc = results['documents'][0][0]

        # 8. Sonucu "Referans Veri" (Ground Truth) ile karşılaştırıyoruz
        if found_id == expected_id:
            print(f"  [PASS] S: '{question[:30]}...' -> Bulunan: '{found_id}'")
            score += 1
        else:
            print(f"  [FAIL] S: '{question[:30]}...' -> Beklenen: '{expected_id}', Bulunan: '{found_id}'")
            print(f"         (Bulunan doküman: '{found_doc[:50]}...')")


    print("\n====== FİNAL DOĞRULUK TESTİ TAMAMLANDI ======")
    print(f"Skor: {score} / {total} doğru")
    if total > 0:
        print(f"Final Getirme Doğruluğu (Retrieval Accuracy): {(score / total) * 100:.1f}%")
    else:
        print("Hiç test senaryosu çalıştırılamadı.")

else:
    print("HATA: Fine-tuning işlemi başarısız olduğu veya atlandığı için Cell 10 (Değerlendirme) çalıştırılmadı.")

Tue Nov 11 13:29:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             27W /   70W |    1318MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Sorular üretiliyor:   0%|          | 0/189 [00:00<?, ?it/s]


Uyarı: JSON parse hatası (onarım sonrası): Ar-Ge ekibi, yeni tork anahtar... Hata: Expecting ',' delimiter: line 3 column 3 (char 58)
Alınan Ham Metin: [
  "Yeni tork anahtarının prototipinin hangi aşamada?"
  "Kalıp maliyeti nedir ve bunu kim belirledi?",
  "Ar-Ge ekibi, yeni tork anahtarını tamamladıklarına dair neye karar verdiler?"
]

Uyarı: Cevapta JSON listesi bulunamadı: Nakit akışımızı düzenlemek içi...
Alınan Ham Metin: [
"Çek Ödemeleri'nin gün sayısı değişti. Yeni saat nedir?",
"Nakit akışını düzenlemeyeçeki amaç nedir?",
"Çek Ödemelerinde süre artırıldı. Neden 60 günden 90 güne çıktı?"

Uyarı: JSON parse hatası (onarım sonrası): Bir pensenin 45 operasyonu var... Hata: Expecting ',' delimiter: line 4 column 3 (char 99)
Alınan Ham Metin: [
  "Penesinin toplam operasyonu sayısı nedir?",
  "Hepsi 'sıfır hata' ile denetlenir demek mi?"
  "Operasyon sayıları sıfır hataya göre denetlenmektedir, bu sonuçların niteliği nedir?"
]

Uyarı: Cevapta JSON listesi bulunamadı: Pense üretimi

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dilvinagachanli (dilvinagachanli-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss



✅ Fine-tuning tamamlandı! Yeni model './my-finetuned-model-v1' klasörüne kaydedildi.
Yeni fine-tune edilmiş model yükleniyor...
====== FİNAL DOĞRULUK TESTİ (Fine-Tuned Model ile) BAŞLIYOR ======

--- Adil bir test için veritabanı YENİ fine-tune edilmiş modelle yeniden oluşturuluyor ---
Yeni fine-tune edilmiş veritabanı oluşturuldu.

--- YENİ Fine-Tuned Veritabanı üzerinde 15 adet test senaryosu çalıştırılıyor ---
  [PASS] S: 'Personel servis saati nedir?...' -> Bulunan: 'doc_0'
  [PASS] S: 'Ahmet D.'nin izni ne oldu?...' -> Bulunan: 'doc_4'
  [PASS] S: 'Güney cephesindeki alarm planı...' -> Bulunan: 'doc_27'
  [PASS] S: 'Zeynep Hanım'ın avans talebi n...' -> Bulunan: 'doc_28'
  [PASS] S: 'Kantin sözleşmesi ne oldu?...' -> Bulunan: 'doc_38'
  [PASS] S: 'ISO 9001 denetiminde ne bulund...' -> Bulunan: 'doc_42'
  [PASS] S: 'Yusuf Bey'in babasına ne oldu?...' -> Bulunan: 'doc_51'
  [PASS] S: 'Elif S.'nin kan grubu nedir?...' -> Bulunan: 'doc_35'
  [PASS] S: '31 CRV 3 çeliğinin maliyeti ne.